<a href="https://colab.research.google.com/github/123Satyajeet123/text-summarizer/blob/main/Conversation_Summarization_Model_Google_Pegasus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Jul 18 07:00:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 92.2 MB/s eta 0:00:

In [3]:
!pip install --upgrade accelerate
!pip uninstall  -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 10.1 MB/s eta 0:00:00
Found existing installation: transformers 4.30.2
Uninstalling transformers-4.30.2:
  Successfully uninstalled transformers-4.30.2
Found existing installation: accelerate 0.21.0
Uninstalling accelerate-0.21.0:
  Successfully uninstalled accelerate-0.21.0
  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
  Using cached accelerate-0.21.0-py3-none-any.whl (244 kB)


In [25]:
from transformers import pipeline,set_seed, AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset,load_from_disk,load_metric
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [27]:
dataset = load_dataset("samsum")

  0%|          | 0/3 [00:00<?, ?it/s]

In [28]:
dataset

NameError: ignored

In [9]:
dataset['train']

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [10]:
dataset['train'].features

{'id': Value(dtype='string', id=None),
 'dialogue': Value(dtype='string', id=None),
 'summary': Value(dtype='string', id=None)}

In [17]:
dataset['train'].features['dialogue']

Value(dtype='string', id=None)

In [20]:
split_lengths = [len(dataset[split]) for split in dataset]
print("split_length",split_lengths)
print("features",dataset['train'].column_names)
print("dialogue")
print(dataset['test'][1]['dialogue'])
print("summary")
print(dataset['test'][1]['summary'])

split_length [14732, 819, 818]
features ['id', 'dialogue', 'summary']
dialogue
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)
summary
Eric and Rob are going to watch a stand-up on youtube.


In [31]:
def convert_to_features(batch):
  input_encodings = tokenizer(batch['dialogue'],max_length = 1024,truncation=True)
  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(batch['summary'],max_length = 128, truncation=True)

  return{
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
  }

In [32]:
dataset_pt = dataset.map(convert_to_features,batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [33]:
dataset_pt

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [35]:
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model = model_pegasus)

In [39]:
from transformers import Trainer, TrainingArguments


In [40]:
trainingArguments = TrainingArguments(
    output_dir = "pegasus",num_train_epochs = 1,warmup_steps = 500,
    per_device_train_batch_size = 1, per_device_eval_batch_size = 1,
    weight_decay=0.01,logging_steps = 10,
    evaluation_strategy = "steps",eval_steps = 500,save_steps = 1e6,
    gradient_accumulation_steps = 16
)

In [41]:
trainer = Trainer(model = model_pegasus,args= trainingArguments,
                  tokenizer= tokenizer,data_collator = seq2seq_data_collator,
                  train_dataset = dataset_pt['train'],
                  eval_dataset = dataset_pt['validation']
                  )

In [42]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,1.690900,1.484756


TrainOutput(global_step=920, training_loss=1.8256099980810414, metrics={'train_runtime': 3233.4579, 'train_samples_per_second': 4.556, 'train_steps_per_second': 0.285, 'total_flos': 5526961323663360.0, 'train_loss': 1.8256099980810414, 'epoch': 1.0})

In [43]:
def generate_batch_sized_chunks(elements,batch_size):
   for i in range(0, len(elements), batch_size):
        yield elements[i:i + batch_size]


In [44]:
def calculate_metric_on_test_ds(dataset,metric,model,tokenizer,batch_size=16,device=device,
                                column_text='article',
                                column_summary='highlights'):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text],batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary],batch_size))

    for article_batch,target_batch in tqdm(
        zip(article_batches,target_batches),total=len(article_batches)):

        inputs = tokenizer(article_batch,max_length = 1024,truncation=True,
                           padding="max_length",return_tensors = 'pt')
        summaries = model.generate(input_ids = inputs['input_ids'].to(device),
                                   attention_mask = inputs['attention_mask'].to(device),
                                   length_penalty = 0.8,num_beams = 8, max_length = 128)

        decoded_summaries = [tokenizer.decode(s,skip_special_tokens = True,
                             clean_up_tokenization_spaces=True)
              for s in summaries]
        decoded_summaries = [d.replace(""," ") for d in decoded_summaries]

        metric.add_batch(predictions=decoded_summaries,references = target_batch)

    score = metric.compute()
    return score


In [45]:
rouge_names = ['rouge1','rouge2','rougeL','rougeLsum']
rouge_metric = load_metric('rouge')

<ipython-input-45-0594f0ee42f1>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')


In [47]:
score = calculate_metric_on_test_ds(dataset['test'],rouge_metric,trainer.model,tokenizer,batch_size =2,column_text="dialogue",column_summary="summary")
rouge_dict= dict((rn,score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict,index=[f"pegasus"])

100%|██████████| 410/410 [13:36<00:00,  1.99s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.018561,0.000322,0.018434,0.018469


No charts were generated by quickchart


In [48]:
model_pegasus.save_pretrained("my-pegasus-samsum-model")

In [49]:
tokenizer.save_pretrained('tokenizer')

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [52]:
#predictions
gen_kwargs = {"length_penalty":0.8,"num_beams":8,"max_length":128}

sample_text = dataset['test'][0]['dialogue']
reference = dataset['test'][0]['summary']

pipe = pipeline('summarization',model = 'my-pegasus-samsum-model',tokenizer = tokenizer)

print("Dialogue:")
print(sample_text)

print("refernce summary:")
print(reference)

Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
refernce summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


In [53]:
print("Model Summary")
print(pipe(sample_text,**gen_kwargs)[0]['summary_text'])

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Model Summary
Amanda can't find Betty's number. Larry called Betty's last time they were at the park together. Hannah wants Amanda to text him.
